In [1]:
%matplotlib notebook

import os
import time
import numpy as np
import matplotlib as mpl

from collections import OrderedDict
from importlib import reload
from matplotlib import pyplot as plt

In [2]:
import qcodes as qc
from qcodes import load_by_id
from qcodes import validators
from qcodes.dataset.measurements import Measurement
from qcodes.instrument.base import Instrument
from qcodes.instrument.parameter import Parameter

In [3]:
from pytopo.qctools.instruments import create_inst
from pytopo.qctools.dataset2 import select_experiment

from plottr import client
from plottr.qcodes_dataset import QcodesDatasetSubscriber

from pytopo.mplplots.init_nb_plotting import *

qc.config['core']['db_location'] = r"d:/OneDrive/BF1/Data/experiments.db"

In [57]:
from qcodes.instrument_drivers.QuTech.IVVI import IVVI
ivvi = create_inst(IVVI, "ivvi", "ASRL5::INSTR")

from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
key1 = create_inst(Keysight_34465A, "key1", "TCPIP::169.254.4.62::INSTR", force_new_instance=True)
key2 = create_inst(Keysight_34465A, "key2", "TCPIP::169.254.4.61::INSTR", force_new_instance=True)
# key3 = create_inst(Keysight_34465A, "key3", "TCPIP::169.254.21.92::INSTR", force_new_instance=True)
# key4 = create_inst(Keysight_34465A, "key4", "TCPIP::169.254.130.42::INSTR", force_new_instance=True)

from qcodes.instrument_drivers.stanford_research.SR860 import SR860
lockin1 = create_inst(SR860, "lockin1", "GPIB0::3::INSTR")
# lockin2 = create_inst(SR860, "lockin2", "GPIB0::4::INSTR")
# lockin3 = create_inst(SR860, "lockin3", "GPIB0::5::INSTR")

for i in range(1,16):
    ivvi.parameters['dac{}'.format(i)].set_step(0.5)
    ivvi.parameters['dac{}'.format(i)].set_delay(0.001)
    
ivvi.dac1.set_delay(0.001)
ivvi.dac1.set_step(20)

ivvi.dac2.set_delay(0.001)
ivvi.dac2.set_step(20)

ivvi.dac3.set_delay(0.001)
ivvi.dac3.set_step(2)

ivvi.dac4.set_delay(0.001)
ivvi.dac4.set_step(20)

ivvi.dac5.set_delay(0.001)
ivvi.dac5.set_step(2)

ivvi.dac6.set_delay(0.001)
ivvi.dac6.set_step(2)

ivvi.dac7.set_delay(0.001)
ivvi.dac7.set_step(2)

ivvi.dac8.set_delay(0.001)
ivvi.dac8.set_step(2)

ivvi.dac9.set_delay(0.001)
ivvi.dac9.set_step(2)

ivvi.dac10.set_delay(0.001)
ivvi.dac10.set_step(2)

ivvi.dac11.set_delay(0.001)
ivvi.dac11.set_step(2)

ivvi.dac12.set_delay(0.001)
ivvi.dac12.set_step(2)

ivvi.dac13.set_delay(0.001)
ivvi.dac13.set_step(2)

from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430,AMI430_3D
ami_x = create_inst(AMI430, "AMI430_x", "169.254.117.25", port = 7180, has_current_rating=True)
ami_y = create_inst(AMI430, "AMI430_y", "169.254.67.160", port = 7180, has_current_rating=True)
ami_z = create_inst(AMI430, "AMI430_z", "169.254.168.239", port = 7180, has_current_rating=True)

#ramp_limit_multiplier = 0.6
#ami_x.current_ramp_limit(0.1742*ramp_limit_multiplier)
#ami_x.coil_constant(0.01491)
#ami_x.current_rating(67.07)

#ami_y.current_ramp_limit(0.1762*ramp_limit_multiplier)
#ami_y.coil_constant(0.01490)
#ami_y.current_rating(67.11)

#ami_z.current_ramp_limit(0.0756*ramp_limit_multiplier)
#ami_z.coil_constant(0.0834)
#ami_z.current_rating(71.94)

ami = create_inst(AMI430_3D, "AMI430", ami_x, ami_y, ami_z, 2)

Instrument ivvi already exists.
Instrument key1 already exists.
Connected to: Keysight Technologies 34465A (serial:MY57503135, firmware:A.02.14-02.40-02.14-00.49-03-01) in 0.13s
Instrument key2 already exists.
Connected to: Keysight Technologies 34465A (serial:MY57503556, firmware:A.02.14-02.40-02.14-00.49-03-01) in 0.13s
Instrument lockin1 already exists.


d:\onedrive\bf1\code\qcodes\qcodes\instrument\parameter.py:518: UserWarning: set_step is deprecated use step property as in `inst.step = stepvalue` instead
  "set_step is deprecated use step property as in `inst.step = "
d:\onedrive\bf1\code\qcodes\qcodes\instrument\parameter.py:530: UserWarning: set_delay is deprecated use inter_delay or post_delay property as in `inst.inter_delay = delayvalue` instead
  "set_delay is deprecated use inter_delay or post_delay property "


Connected to: AMERICAN MAGNETICS INC. 430 (serial:170025, firmware:2.50) in 1.28s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170026, firmware:2.50) in 1.25s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170024, firmware:2.50) in 1.18s


In [92]:
class ConversionParameter(Parameter):
    
    def __init__(self, name, src_param, get_conv, set_conv=None, **kw):
        super().__init__(name, **kw)
        self.src_param = src_param
        self.get_conv = get_conv
        self.set_conv = set_conv
    
    def get_raw(self):
        return self.get_conv(self.src_param())
    
    def set_raw(self, value):
        if self.set_conv is None:
            raise NotImplementedError("No set conversion implemented.")
        
        return self.src_param(self.set_conv(value))
    

class IVVISetup(Instrument):
    
    def __init__(self, name, ivvi, **kw):
        super().__init__(name, **kw)
        
        self.ivvi = ivvi
        
        self.add_parameter('voltage_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=1e-3)
        self.add_parameter('current_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=1e-7)
        self.add_parameter('iso_in_multiplier', get_cmd=None, set_cmd=None,
                           initial_value=1e-2)
        self.add_parameter('gate_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=15)
        
        self.add_parameter('M1b_gain', get_cmd=None, set_cmd=None, initial_value=1e6)
        self.add_parameter('M1b_mode', get_cmd=None, set_cmd=None, 
                           vals=validators.Enum('low_noise', 'low_R_in'), 
                           initial_value='low_noise')
        self.add_parameter('M1b_R_in', get_cmd=self._get_M1b_R_in)
        
        self.add_parameter('M2m_Vgain', get_cmd=None, set_cmd=None, initial_value=1e3)

        for i in range(1, self.ivvi._numdacs + 1):       
            self.add_parameter('dac{}'.format(i), unit='V',
                               get_cmd=self.ivvi._gen_ch_get_func(self._get_dac, i),
                               set_cmd=self.ivvi._gen_ch_set_func(self._set_dac, i))
            
    def _get_M1b_R_in(self):
        if self.M1b_mode() == 'low_noise':
            return 2e3 + 1e-3 * self.M1b_gain()
        elif self.M1b_mode() == 'low_R_in':
            return 2e3 + 1e-4 * self.M1b_gain()
            
    def _get_dac(self, ch):
        if ch in [7, 8, 9, 10, 11, 12, 13]:
            return self.ivvi.set('dac{}'.format(ch)) * 1e-3 * self.gate_multiplier()
        else:
            return self.ivvi.get('dac{}'.format(ch)) * 1e-3 * self.voltage_multiplier()
    
    def _set_dac(self, ch, val):
        if ch in [7, 8, 9, 10, 11, 12, 13]:
            return self.ivvi.set('dac{}'.format(ch), val * 1e3 / self.gate_multiplier())
        else:
            return self.ivvi.set('dac{}'.format(ch), val * 1e3 / self.voltage_multiplier())

In [164]:
ivvi_setup = create_inst(IVVISetup, 'ivvi_setup', ivvi=ivvi, force_new_instance=True)
ivvi_setup.voltage_multiplier(10e-3)
ivvi_setup.gate_multiplier(1)


dev1_line_r = 10e3 + 3.4e3 #1.7e3 from Qfilter, 5e3 for DC-only on PCB, 10e3 for RF on PCB, 3e3 for 8x MUX bias-T
# dev1_line_r = 0


dev1_rs = dev1_line_r + ivvi_setup.M1b_R_in()

dev1_Vac = lockin1.amplitude()*ivvi_setup.iso_in_multiplier()*ivvi_setup.voltage_multiplier()

ivvi_setup.add_parameter('i_measurement', parameter_class=ConversionParameter, unit='A',
                         src_param=key1.volt, get_conv=lambda x: x / ivvi_setup.M1b_gain())
ivvi_setup.add_parameter('ig_measurement', parameter_class=ConversionParameter, unit='A',
                         src_param=key2.volt, get_conv=lambda x: x / ivvi_setup.M1b_gain())

ivvi_setup.add_parameter('g_measurement', parameter_class=ConversionParameter, unit='2e2/h',
                         src_param=lockin1.X, get_conv=lambda x: 12906/(dev1_Vac*ivvi_setup.M1b_gain()/x - dev1_rs))

Instrument ivvi_setup already exists.


In [165]:
ivvi_setup.M1b_gain(1e7)
ivvi_setup.M1b_mode('low_noise')

key1.NPLC(1)
key2.NPLC(1)

print('R_in: ', ivvi_setup.M1b_R_in())
print('Rs_1: ', dev1_rs)

R_in:  12000.0
Rs_1:  16400.0


# IV

In [68]:
exp = select_experiment('IV', 'Cq334R')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac4, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac4, ])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac4, ])


srcvals = np.linspace(-10e-3, 10e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac4(s)
        datasaver.add_result((ivvi_setup.dac4, s), 
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 466


# Pinchoff

In [169]:
exp = select_experiment('T1', 'Cq334M')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac8)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac8, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac8, ])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac8, ])

lockin1.amplitude(0.05)
ivvi_setup.dac4(1e-3)

srcvals = np.linspace(0, -0.3, 301)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac8(s)
        datasaver.add_result((ivvi_setup.dac8, s), 
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 540


In [170]:
exp = select_experiment('T1', 'Cq334M')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac8)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac8, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac8, ])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac8, ])

lockin1.amplitude(0.05)
ivvi_setup.dac4(1e-3)

srcvals = np.linspace(-0.3, 0, 301)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in srcvals:
        ivvi_setup.dac8(s)
        datasaver.add_result((ivvi_setup.dac8, s), 
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 541


In [174]:
ivvi.set_dacs_zero()

# Vb vs gate

In [104]:
ami_x.pause()
ami_y.pause()
ami_z.pause()
ami.spherical([0.0,90,0])

In [97]:
exp = select_experiment('T2', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac6, ivvi_setup.dac4])

lockin1.amplitude(0.1)
ivvi_setup.dac8(-0.35)
ivvi_setup.dac11(-0.37)

r2 = np.linspace(0, 2e-3, 41)
bias_values = np.linspace(-.5e-3, .5e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in r2:
        ivvi_setup.dac6(s1)
        for s2 in bias_values:
            ivvi_setup.dac4(s2)
            datasaver.add_result((ivvi_setup.dac6, s1),
                                 (ivvi_setup.dac4, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 485


# Vb vs R1

In [172]:
exp = select_experiment('T2', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac4])

lockin1.amplitude(0.05)
ivvi_setup.dac7(-0.27) #t1
ivvi_setup.dac8(-0.24) #t2

r1 = np.linspace(0, 5e-3, 201)
bias_values = np.linspace(-.5e-3, .5e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in r1:
        ivvi_setup.dac5(s1)
        for s2 in bias_values:
            ivvi_setup.dac4(s2)
            datasaver.add_result((ivvi_setup.dac5, s1),
                                 (ivvi_setup.dac4, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 543


In [109]:
exp = select_experiment('T2', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac4])

lockin1.amplitude(0.1)
ivvi_setup.dac7(-0.33)
ivvi_setup.dac4(0.0e-3)
ivvi_setup.dac6(0.0e-3)
ivvi_setup.dac8(-0.345)
ivvi_setup.dac11(-0.365)

pg = np.linspace(0, 1.5e-3, 75)
bias_values = np.linspace(-.5e-3, .5e-3, 151)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in pg:
        ivvi_setup.dac5(s1)
        for s2 in bias_values:
            ivvi_setup.dac4(s2)
            datasaver.add_result((ivvi_setup.dac5, s1),
                                 (ivvi_setup.dac4, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 496


# Gate vs gate

In [20]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac7)
meas.register_parameter(ivvi_setup.dac8)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac7, ivvi_setup.dac8])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac7, ivvi_setup.dac8])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac7, ivvi_setup.dac8])

lockin1.amplitude(0.1)
ivvi_setup.dac4(1e-3)

t2 = np.linspace(-0.32, -0.36, 50)
t3 = np.linspace(-0.32, -0.36, 50)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in t2:
        ivvi_setup.dac7(s1)
        for s2 in t3:
            ivvi_setup.dac8(s2)
            datasaver.add_result((ivvi_setup.dac7, s1),
                                 (ivvi_setup.dac8, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 442


# double dot

In [121]:
ami_x.pause()
ami_y.pause()
ami_z.pause()
ami.spherical([0,0,90])

In [122]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])

lockin1.amplitude(0.05)

ivvi_setup.dac7(-0.335)
ivvi_setup.dac4(0.05e-3)
ivvi_setup.dac8(-0.35)
ivvi_setup.dac11(-0.37)

pg = np.linspace(0, 1e-3, 51)
r2 = np.linspace(0, -2e-3, 51)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in pg:
        ivvi_setup.dac5(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac5, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 504


In [138]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac10)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac10, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac10, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac10, ivvi_setup.dac6])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.35)
ivvi_setup.dac7(-0.33)
ivvi_setup.dac4(0.0e-3)
ivvi_setup.dac5(0.0e-3)
ivvi_setup.dac8(-0.34)
ivvi_setup.dac11(-0.365)

r1 = np.linspace(0, 2e-3, 32)
r2 = np.linspace(0, -2e-3, 101)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)

    for s1 in r1:
        ivvi_setup.dac10(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac10, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 519


In [150]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac13)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.35) #T1
ivvi_setup.dac7(-0.33) #T2
ivvi_setup.dac4(0.0e-3) #bias
ivvi_setup.dac5(0.0e-3) #G
ivvi_setup.dac8(-0.34) #T3
ivvi_setup.dac11(-0.365) #T11
ivvi_setup.dac10(0) #R1

p1 = np.linspace(0e-3, 150e-3, 25)
r2 = np.linspace(-2e-3, 0, 51)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)

    for s1 in p1:
        ivvi_setup.dac13(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac13, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 531


In [155]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac13)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.35) #T1
ivvi_setup.dac7(-0.33) #T2
ivvi_setup.dac4(0.0e-3) #bias
ivvi_setup.dac5(0.0e-3) #G
ivvi_setup.dac8(-0.335) #T3
ivvi_setup.dac11(-0.365) #T11
ivvi_setup.dac10(0) #R1

p1 = np.linspace(-300e-3, 0e-3, 31)
r2 = np.linspace(-2e-3, 0, 51)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)

    for s1 in p1:
        ivvi_setup.dac13(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac13, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 535


KeyboardInterrupt: 

In [145]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac13)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.35) #T1
ivvi_setup.dac7(-0.325) #T2
ivvi_setup.dac4(0.0e-3) #bias
ivvi_setup.dac5(0.0e-3) #G
ivvi_setup.dac8(-0.335) #T3
ivvi_setup.dac11(-0.365) #T11
ivvi_setup.dac10(0) #R1

p1 = np.linspace(0e-3, 150e-3, 151)
r2 = np.linspace(-2e-3, 0, 201)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)

    for s1 in p1:
        ivvi_setup.dac13(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac13, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 526


In [146]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac13)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.35) #T1
ivvi_setup.dac7(-0.32) #T2
ivvi_setup.dac4(0.0e-3) #bias
ivvi_setup.dac5(0.0e-3) #G
ivvi_setup.dac8(-0.33) #T3
ivvi_setup.dac11(-0.365) #T11
ivvi_setup.dac10(0) #R1

p1 = np.linspace(0e-3, 150e-3, 151)
r2 = np.linspace(-2e-3, 0, 201)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)

    for s1 in p1:
        ivvi_setup.dac13(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac13, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 527


In [147]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac13)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.35) #T1
ivvi_setup.dac7(-0.3) #T2
ivvi_setup.dac4(0.0e-3) #bias
ivvi_setup.dac5(0.0e-3) #G
ivvi_setup.dac8(-0.32) #T3
ivvi_setup.dac11(-0.365) #T11
ivvi_setup.dac10(0) #R1

p1 = np.linspace(0e-3, 150e-3, 151)
r2 = np.linspace(-2e-3, 0, 201)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)

    for s1 in p1:
        ivvi_setup.dac13(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac13, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 528


In [148]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac13)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac13, ivvi_setup.dac6])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.35) #T1
ivvi_setup.dac7(-0.3) #T2
ivvi_setup.dac4(0.0e-3) #bias
ivvi_setup.dac5(0.0e-3) #G
ivvi_setup.dac8(-0.3) #T3
ivvi_setup.dac11(-0.365) #T11
ivvi_setup.dac10(0) #R1

p1 = np.linspace(0e-3, 150e-3, 151)
r2 = np.linspace(-2e-3, 0, 201)

with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)

    for s1 in p1:
        ivvi_setup.dac13(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac13, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 529


In [ ]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])

lockin1.amplitude(0.05)

ivvi_setup.dac7(-0.335)
ivvi_setup.dac4(0.05e-3)
ivvi_setup.dac8(-0.35)
ivvi_setup.dac11(-0.37)

pg = np.linspace(0, 1e-3, 101)
r2 = np.linspace(0, -2e-3, 101)
fields = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
for field in fields:
    ami.field(field)
    with meas.run() as datasaver:
        datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                    min_wait=0, min_count=1)

        for s1 in pg:
            ivvi_setup.dac5(s1)
            for s2 in r2:
                ivvi_setup.dac6(s2)
                datasaver.add_result((ivvi_setup.dac5, s1),
                                     (ivvi_setup.dac6, s2), 
                                     (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                     (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                     (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

In [108]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ivvi_setup.dac6)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac5, ivvi_setup.dac6])

lockin1.amplitude(0.1)

ivvi_setup.dac7(-0.33)
ivvi_setup.dac4(0.0e-3)
ivvi_setup.dac8(-0.345)
ivvi_setup.dac11(-0.365)

pg = np.linspace(0, 2e-3, 201)
r2 = np.linspace(0, 4e-3, 201)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in pg:
        ivvi_setup.dac5(s1)
        for s2 in r2:
            ivvi_setup.dac6(s2)
            datasaver.add_result((ivvi_setup.dac5, s1),
                                 (ivvi_setup.dac6, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 495


# Plunger sweep

In [48]:
exp = select_experiment('G', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac5, ])

lockin1.amplitude(0.1)
ivvi_setup.dac7(-0.335)
ivvi_setup.dac8(-0.335)
ivvi_setup.dac4(0e-3)

pg = np.linspace(0, 1e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in pg:
        ivvi_setup.dac5(s)
        datasaver.add_result((ivvi_setup.dac5, s), 
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 462


# R2 sweep

In [134]:
exp = select_experiment('G', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac10)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac10, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac10, ])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac10, ])

lockin1.amplitude(0.05)
ivvi_setup.dac9(-0.36)
ivvi_setup.dac7(-0.33)
ivvi_setup.dac4(0)

r1 = np.linspace(0, 10e-3, 160)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in r1:
        ivvi_setup.dac10(s)
        datasaver.add_result((ivvi_setup.dac10, s), 
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 517


# T3 vs T4

In [73]:
exp = select_experiment('T2_vs_T3', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac11)
meas.register_parameter(ivvi_setup.dac8)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac11, ivvi_setup.dac8])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac11, ivvi_setup.dac8])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac11, ivvi_setup.dac8])

lockin1.amplitude(0.1)
ivvi_setup.dac4(1e-3)

t4 = np.linspace(-0.35, -0.39, 25)
t3 = np.linspace(-0.32, -0.38, 50)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in t4:
        ivvi_setup.dac11(s1)
        for s2 in t3:
            ivvi_setup.dac8(s2)
            datasaver.add_result((ivvi_setup.dac11, s1),
                                 (ivvi_setup.dac8, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 469


KeyboardInterrupt: 

# Bias vs theta

In [112]:
ami_x.pause()
ami_y.pause()
ami_z.pause()
ami.spherical([0.5,0,90])

In [113]:
exp = select_experiment('theta_align', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ami.theta)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ami.theta, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ami.theta, ivvi_setup.dac4])

lockin1.amplitude(0.1)

theta_vals = np.linspace(0, 180, 90)
bias_values = np.linspace(-.4e-3, .4e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac4(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac4, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 498


In [114]:
ami_x.pause()
ami_y.pause()
ami_z.pause()
ami.spherical([0.75,180,90])

In [115]:
exp = select_experiment('theta_align', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac4)
meas.register_parameter(ami.theta)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ami.theta, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.theta, ivvi_setup.dac4])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ami.theta, ivvi_setup.dac4])

lockin1.amplitude(0.1)

theta_vals = np.linspace(180, 0, 90)
bias_values = np.linspace(-.4e-3, .4e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in theta_vals:
        ami.theta(s1)
        for s2 in bias_values:
            ivvi_setup.dac4(s2)
            datasaver.add_result((ami.theta, s1),
                                 (ivvi_setup.dac4, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 499


In [128]:
exp = select_experiment('G', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ivvi_setup.dac5, ])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ivvi_setup.dac5, ])

ivvi_setup.dac7(-0.33)
ivvi_setup.dac4(0.0e-3)
ivvi_setup.dac8(-0.345)
ivvi_setup.dac11(-0.365)
ivvi_setup.dac6(-1e-3)

pg = np.linspace(0, 2e-3, 101)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s in pg:
        ivvi_setup.dac5(s)
        datasaver.add_result((ivvi_setup.dac5, s), 
                             (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                             (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                             (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 512


In [136]:
ami_x.pause()
ami_y.pause()
ami_z.pause()
ami.spherical([0,0,90])

In [131]:
exp = select_experiment('theta_align', 'Cq334L')

meas = Measurement(exp=exp)
meas.write_period = 1
meas.register_parameter(ivvi_setup.dac5)
meas.register_parameter(ami.field)
meas.register_parameter(ivvi_setup.i_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.ig_measurement, setpoints=[ami.field, ivvi_setup.dac5])
meas.register_parameter(ivvi_setup.g_measurement, setpoints=[ami.field, ivvi_setup.dac5])


lockin1.amplitude(0.05)
ivvi_setup.dac7(-0.33)
ivvi_setup.dac4(0.0e-3)
ivvi_setup.dac8(-0.345)
ivvi_setup.dac11(-0.365)
ivvi_setup.dac6(-1e-3)

field_vals = np.linspace(1, 0, 51)
pg = np.linspace(0, 4e-3, 401)
with meas.run() as datasaver:
    datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
                                min_wait=0, min_count=1)
    
    for s1 in field_vals:
        ami.field(s1)
        for s2 in pg:
            ivvi_setup.dac5(s2)
            datasaver.add_result((ami.field, s1),
                                 (ivvi_setup.dac5, s2), 
                                 (ivvi_setup.i_measurement, ivvi_setup.i_measurement()),
                                 (ivvi_setup.ig_measurement, ivvi_setup.ig_measurement()),
                                 (ivvi_setup.g_measurement, ivvi_setup.g_measurement()))

Starting experimental run with id: 515
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
T

Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during sending!
Timeout during s